# Video Frame Differencing
Sample every Nth frame from a video, then compute the difference between each sampled frame and the one K samples earlier. Save both the sampled RGB frames and the diff images.

In [6]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Repo root (two levels up from analysis/tutorials/)
ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

In [7]:
def extract_frames(video_path, sample_rate=30, diff_gap=1):
    """
    Extract every Nth frame from a video, plus the frame K real frames before each.

    Returns
    -------
    sampled : list of (frame_index, RGB numpy array)  – the Nth-frame snapshots
    lookback : dict  {video_frame_index: RGB numpy array} – the K-back frames
               (only for indices >= diff_gap)
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Cannot open video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Video: {total_frames} frames, {fps:.1f} fps, sampling every {sample_rate} frames")

    # Figure out which video indices we need to grab
    sampled_indices = set(range(0, total_frames, sample_rate))
    lookback_indices = {idx - diff_gap for idx in sampled_indices if idx - diff_gap >= 0}
    need_indices = sampled_indices | lookback_indices

    sampled = []
    lookback = {}
    idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if idx in need_indices:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if idx in sampled_indices:
                sampled.append((idx, rgb_frame))
            if idx in lookback_indices:
                lookback[idx] = rgb_frame
        idx += 1

    cap.release()
    print(f"Extracted {len(sampled)} sampled frames, {len(lookback)} lookback frames (K={diff_gap})")
    return sampled, lookback

In [8]:
# ============================================================
# Diff metric – change this cell to swap the differencing method
# ============================================================
# Input : two RGB uint8 numpy arrays of the same shape (H, W, 3)
# Output: a numpy array suitable for saving as an image
# ============================================================

def compute_diff(frame_a, frame_b):
    """Absolute pixel difference between two RGB frames.
    
    Returns a uint8 RGB image where each pixel is |frame_a - frame_b|.
    """
    return np.abs(frame_a.astype(np.int16) - frame_b.astype(np.int16)).astype(np.uint8)

In [9]:
def process_video_diff(video_path, output_dir, sample_rate=30, diff_gap=1):
    """
    Full pipeline: extract sampled frames, compute diffs, save RGB + diff images.

    Parameters
    ----------
    video_path : str   – path to input video
    output_dir : str   – root output directory
    sample_rate : int  – N: capture every Nth video frame
    diff_gap : int     – K: for each sampled frame at video index F,
                         diff against the real video frame at index F-K

    Output structure:
        <output_dir>/<video_name>-rgb/frame_000001.png, ...
        <output_dir>/<video_name>-diff/diff_000001.png, ...
    """
    video_name = Path(video_path).stem

    rgb_dir  = os.path.join(output_dir, f"{video_name}-rgb")
    diff_dir = os.path.join(output_dir, f"{video_name}-diff")
    os.makedirs(rgb_dir, exist_ok=True)
    os.makedirs(diff_dir, exist_ok=True)

    sampled, lookback = extract_frames(video_path, sample_rate, diff_gap)

    # Save all sampled RGB frames
    for i, (frame_idx, rgb_frame) in enumerate(sampled):
        fname = f"frame_{i+1:06d}.png"
        rgb_path = os.path.join(rgb_dir, fname)
        plt.imsave(rgb_path, rgb_frame)

    print(f"Saved {len(sampled)} RGB frames to {rgb_dir}")

    # Compute and save diffs: for each sampled frame F, diff with frame F-K
    diff_count = 0
    skipped = 0
    for i, (frame_idx, curr_frame) in enumerate(sampled):
        lookback_idx = frame_idx - diff_gap
        if lookback_idx < 0:
            skipped += 1
            continue

        prev_frame = lookback[lookback_idx]
        diff_img = compute_diff(curr_frame, prev_frame)

        fname = f"diff_{i+1:06d}.png"
        diff_path = os.path.join(diff_dir, fname)
        plt.imsave(diff_path, diff_img)

        diff_count += 1
        print(f"  [{diff_count}/{len(sampled) - skipped}] diff( frame {frame_idx} , frame {lookback_idx} ) -> {fname}")

    print(f"\nDone! RGB: {rgb_dir}  |  Diff: {diff_dir}")
    print(f"  {len(sampled)} sampled frames, {diff_count} diff images (N={sample_rate}, K={diff_gap})")

# Run Video Frame Differencing
Set your input video, output directory, sample rate (N), and diff gap (K) below.

In [10]:
# ---- Hyperparameters (paths relative to repo root) ----
input_video_path = "analysis/data/depth/surgery_video.mp4"
output_dir       = "analysis/outputs/diff/"
sample_rate      = 30     # N: sample every Nth frame
diff_gap         = 1      # K: diff between sampled frame i and sampled frame i-K
# --------------------------------------------------------

input_video_path = os.path.join(ROOT, input_video_path)
output_dir       = os.path.join(ROOT, output_dir)

process_video_diff(input_video_path, output_dir, sample_rate, diff_gap)

Video: 1638 frames, 24.0 fps, sampling every 30 frames


Extracted 55 sampled frames, 54 lookback frames (K=1)
Saved 55 RGB frames to /home/navlab/sukeerth/Tree/TreeHacks/treehacks-2026/analysis/outputs/diff/surgery_video-rgb
  [1/54] diff( frame 30 , frame 29 ) -> diff_000002.png
  [2/54] diff( frame 60 , frame 59 ) -> diff_000003.png
  [3/54] diff( frame 90 , frame 89 ) -> diff_000004.png
  [4/54] diff( frame 120 , frame 119 ) -> diff_000005.png
  [5/54] diff( frame 150 , frame 149 ) -> diff_000006.png
  [6/54] diff( frame 180 , frame 179 ) -> diff_000007.png
  [7/54] diff( frame 210 , frame 209 ) -> diff_000008.png
  [8/54] diff( frame 240 , frame 239 ) -> diff_000009.png
  [9/54] diff( frame 270 , frame 269 ) -> diff_000010.png
  [10/54] diff( frame 300 , frame 299 ) -> diff_000011.png
  [11/54] diff( frame 330 , frame 329 ) -> diff_000012.png
  [12/54] diff( frame 360 , frame 359 ) -> diff_000013.png
  [13/54] diff( frame 390 , frame 389 ) -> diff_000014.png
  [14/54] diff( frame 420 , frame 419 ) -> diff_000015.png
  [15/54] diff( fram